In [23]:
import pandas as pd

In [24]:
pd.__version__

'1.4.2'

In [25]:
!pip install pyarrow

In [26]:
!pwd

/workspaces/mlops-zoomcamp/01-intro/notebooks


In [27]:
df = pd.read_parquet('/workspaces/mlops-zoomcamp/01-intro/data/yellow_tripdata_2023-01.parquet')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [29]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [30]:
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

In [31]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [32]:
((df.duration >= 1) & (df.duration <= 60)).mean()

0.9812202822125979

In [33]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [34]:
categorical = ['PULocationID', 'DOLocationID']
#numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [35]:
from sklearn.feature_extraction import DictVectorizer

In [38]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [39]:
X_train.shape

(3009173, 515)

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [43]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605